In [1]:
import sys
sys.path.append("/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs")

import numpy as np
import pandas as pd 
from time import time

from tools import functions, constants  # type: ignore 
from tools import functions_readfiles as readfiles # type: ignore
from tools.functions_create_galactic_properties import sfr_calculator, halo_mass, mass_average # type: ignore

from astropy.io import fits

import matplotlib.pyplot as plt 
import matplotlib
plt.style.use("seaborn-poster")

# Functions

In [ ]:
# def get_galactic_properties_for_a_galaxy(galaxy_name:str, galaxy_type:str, redshift:str, directory_name:str):
    
#     print(f"--------------  {galaxy_name}  --------------")
    
#     start = time()

    
#     # Read gas particles
    
#     file_name_lines = "L_line_smoothingLength_hybridInterpolator_flux2Luminosity.txt"
#     gas, lines = readfiles.read_interpolated_Lline(
#         galaxy_name = galaxy_name, 
#         galaxy_type = galaxy_type, 
#         redshift = redshift, 
#         directory_name = directory_name, 
#         file_name = file_name_lines
#     )

#     other_properties_file_name = "otherProperties_smoothingLength_hybridInterpolator.txt"
#     gas_other_properties = readfiles.read_otherProperties(
#             galaxy_name = galaxy_name, 
#             galaxy_type = galaxy_type, 
#             redshift = redshift, 
#             directory_name = directory_name, 
#             file_name = other_properties_file_name,
#         )   

#     # Merge gas particles 
#     ## TODO: In the future change x,y,z with the index and merge according to index.
#     merged_columns = ['index', 'fh2', 'fCO']
#     gas = gas.merge(gas_other_properties[merged_columns], how='inner', on=['index'], validate='one_to_one')
        

#     # Read star particles
#     star = readfiles.read_comprehensive_star_particles(galaxy_name, galaxy_type, redshift, directory_name)

    
#     # Read semi_analytical_average_sobolev_smoothingLength.txt
#     semi_analytical_file_name = "semi_analytical_smoothingLength_cf_1.txt"
#     semi_analytical = readfiles.read_semianalytical_file_2(
#         galaxy_name = galaxy_name, 
#         galaxy_type = galaxy_type, 
#         redshift = redshift, 
#         directory_name = directory_name, 
#         file_name = semi_analytical_file_name,
#     )
    
#     # Read sed file 
#     sed = readfiles.read_skirt_sed_file(galaxy_name, galaxy_type, redshift, directory_name = "voronoi_1e6", inclination = "0")        
    
#     ############ Calculate galactic properties 
#     ## sfr
#     sfr_instantaneous = np.sum(gas["sfr"]) 
#     sfr_5Myr = sfr_calculator(star_df = star, within_how_many_Myr = 5)
#     sfr_10Myr = sfr_calculator(star_df = star, within_how_many_Myr = 10)
#     sfr_100Myr = sfr_calculator(star_df = star, within_how_many_Myr = 100)

#     ## gas mass
#     total_gas_mass = np.sum(gas["mass"])
#     total_star_mass = np.sum(star["mass"])

#     ###### Metalicity
    
#     ## Filtering 
    
#     # Find the indices of gas and star particles within 8 kpc from the center
#     R_inner = 8e3 #kpc 
#     indices_gas_inner_galaxy = np.where(np.sqrt(gas['x']**2 + gas['y']**2 + gas['z']**2) < R_inner)[0] 
#     indices_star_inner_galaxy = np.where(np.sqrt(star['x']**2 + star['y']**2 + star['z']**2) < R_inner)[0] 
#     gas_inner_galaxy = gas.iloc[indices_gas_inner_galaxy]
#     star_inner_galaxy = star.iloc[indices_star_inner_galaxy]

#     # Use only the ism particles. Put a condition on density.   
#     gas_hden_higher_than_1eminus2_condition = gas['hden'] > 1e-2
#     gas_hden_higher_than_1 = gas['hden'] > 1e0

#     ## calculation
#     # all gas particles
#     average_gas_metallicity = mass_average(
#         property_array = gas['metallicity'].to_numpy(),
#         mass = gas['mass'].to_numpy(),
#     )

#     average_star_metallicity = mass_average(
#         property_array = star['metallicity'].to_numpy() / constants.solar_metallicity,
#         mass = star['mass'].to_numpy(),
#     )    

#     # inner galaxy
#     gas_metallicity_inner_galaxy = mass_average(
#         property_array = gas_inner_galaxy['metallicity'].to_numpy(),
#         mass = gas_inner_galaxy['mass'].to_numpy(),
#     )

#     star_metallicity_inner_galaxy = mass_average(
#         property_array = star_inner_galaxy['metallicity'].to_numpy() / constants.solar_metallicity,
#         mass = star_inner_galaxy['mass'].to_numpy()
#     )

#     # hden condition
#     filtered_gas = gas[gas_hden_higher_than_1eminus2_condition].copy()
#     gas_metallicity_hden_higher_than_1eminus2 = mass_average(
#         property_array = filtered_gas['metallicity'].to_numpy(),
#         mass = filtered_gas['mass'].to_numpy(),
#     )

#     filtered_gas = gas[gas_hden_higher_than_1].copy()
#     gas_metallicity_hden_higher_than_1 = mass_average(
#         property_array = filtered_gas['metallicity'].to_numpy(),
#         mass = filtered_gas['mass'].to_numpy(),
#     )

#     ## line_luminosities
#     # Krumholz - semi_analytical 
#     semi_analytical_Lco = np.sum(semi_analytical['L_co'])
#     # Cloudy
#     total_line_luminosities = {}
#     for line in lines: 
#         total_line_luminosities[line] = sum(gas[line])
    
#     ## Molecular gas mass
#     h2_mass_semi_anaytical = np.sum(semi_analytical["Mh2"]) # if h2 mass is NaN probably it is zero already.
#     h2_mass_cloudy = np.sum(gas['fh2'] * gas['mass'])

#     # CO mass
#     co_mass_cloudy = np.sum(gas['fCO'] * gas['mass'])

#     ## halo mass
#     Mhalo = halo_mass(
#         galaxy_name = galaxy_name, 
#         galaxy_type = galaxy_type, 
#         redshift = redshift
#     )

#     # NaN indices
#     nan_indices = np.where(np.isnan(gas["L_co_10"]))[0]    

#     #### Calculate infrared luminosity from sed data 
#     # Define the bands 
#     bands = {
#         "all": {
#             "min_wavelength": 1e-6,
#             "max_wavelength": 1e6,    
#         },
#         "fir": {
#             "min_wavelength": 3,
#             "max_wavelength": 1000,
#         }
#     }    

#     ## SED
#     distance = 10 # Mpc 

#     L_fir = functions.calculate_luminosity_from_sed(
#         sed = sed, 
#         min_wavelength = bands['fir']['min_wavelength'], 
#         max_wavelenght = bands['fir']['max_wavelength'], 
#         distance_in_Mpc = distance
#     ) # erg / s

#     L_skirt_full_wavelength_range = functions.calculate_luminosity_from_sed(
#         sed = sed, 
#         min_wavelength = bands['all']['min_wavelength'], 
#         max_wavelenght = bands['all']['max_wavelength'], 
#         distance_in_Mpc = distance
#     ) # erg / s    

#     # Create a dictionary 
#     other_properties = {
#         "name": f"{galaxy_name}",
#         "galaxy_type": f"{galaxy_type}",
#         "redshift": f"{redshift}",
#         "sfr": sfr_instantaneous, # Msolar / year
#         "sfr_5Myr": sfr_5Myr, # Msolar / year
#         "sfr_10Myr": sfr_10Myr, # Msolar / year
#         "sfr_100Myr": sfr_100Myr, # Msolar / year
#         "gas_mass": total_gas_mass, # Msolar
#         "star_mass": total_star_mass, # Msolar
#         "gas_average_metallicity": average_gas_metallicity, # Zsolar
#         "star_average_metallicity": average_star_metallicity, # Zsolar
#         "gas_metallicity_inner_galaxy": gas_metallicity_inner_galaxy, # Zsolar
#         "star_metallicity_inner_galaxy": star_metallicity_inner_galaxy, # Zsolar
#         "h2_mass_semi_analytical": h2_mass_semi_anaytical, # Msolar
#         "h2_mass_cloudy": h2_mass_cloudy, # Msolar
#         "co_mass_cloudy": co_mass_cloudy, # Msolar
#         "halo_mass": Mhalo, # Msolar,
#         "L_fir": L_fir, # erg / s
#         "L_skirt_full_wavelength_range": L_skirt_full_wavelength_range, # erg / s
#         "number_of_NaN_indices": len(nan_indices),
#         "gas_metallicity_hden_higher_than_1eminus2": gas_metallicity_hden_higher_than_1eminus2, # Zsolar
#         "gas_metallicity_hden_higher_than_1": gas_metallicity_hden_higher_than_1, # Zsolar
#         "L_co_10_semi_analytical": semi_analytical_Lco, # K km s^-1 pc^2 
#     }     

#     # Merge two dictionaries
#     galactic_properties = {**other_properties, **total_line_luminosities}

#     stop = time()
    
#     print(f"For {galaxy_name}, it took {round((stop-start)/60, 3)} minutes")

#     return galactic_properties


# def run_for_single_galaxy(galaxy_name, galaxy_type, redshift, base_dir):
#         try:
#             return get_galactic_properties_for_a_galaxy(
#                 galaxy_name=galaxy_name, 
#                 galaxy_type=galaxy_type, 
#                 redshift=redshift, 
#                 base_dir=base_dir
#             )
            
#         except Exception as e:
#             print(f"Exception occured for galaxy {galaxy_name}: \n{e}")    
#             return None


# Main

In [2]:
# galaxy_name="gal8"
# galaxy_type="firebox"

galaxy_name="m12i_res7100_md"
galaxy_type="zoom_in"

redshift="0.0"
directory_name="voronoi_1e6"

In [3]:
save_dir = "/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/python_files/analyze_hden_metallicity_turbulence_isrf_radius/data"

base_fdir = "/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius"
read_dir = f"{base_fdir}/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}"

file_names = {
    "interpolated_lines": "L_line_smoothingLength_hybridInterpolator_flux2Luminosity.txt",
    "otherProperties": "otherProperties_smoothingLength_hybridInterpolator.txt",
    "semi_analytical": "semi_analytical_smoothingLength_cf_1.txt",
    "i0_total_fits": f"{read_dir}/{galaxy_name}_i0_total.fits",
    "skirt_wavelengths": f"{read_dir}/{galaxy_name}_grid_radiationField_wavelengths.dat",
    "write_file_name": f"{save_dir}/galactic_properties_smoothingLength_hybridInterpolator_z{int(float(redshift))}_usingIvalues.csv"
}

In [4]:
print(f"--------------  {galaxy_name}  --------------")

start = time()


# Read gas particles

gas, lines = readfiles.read_interpolated_Lline(
    galaxy_name = galaxy_name, 
    galaxy_type = galaxy_type, 
    redshift = redshift, 
    directory_name = directory_name, 
    file_name = file_names['interpolated_lines']
)

gas_other_properties = readfiles.read_otherProperties(
        galaxy_name = galaxy_name, 
        galaxy_type = galaxy_type, 
        redshift = redshift, 
        directory_name = directory_name, 
        file_name = file_names['otherProperties'],
    )   

# Merge gas particles 
merged_columns = ['index', 'fh2', 'fCO']
gas = gas.merge(gas_other_properties[merged_columns], how='inner', on=['index'], validate='one_to_one')


# Read star particles
star = readfiles.read_comprehensive_star_particles(galaxy_name, galaxy_type, redshift, directory_name)


# Read semi_analytical_average_sobolev_smoothingLength.txt
semi_analytical = readfiles.read_semianalytical_file_2(
    galaxy_name = galaxy_name, 
    galaxy_type = galaxy_type, 
    redshift = redshift, 
    directory_name = directory_name, 
    file_name = file_names['semi_analytical'],
)

# Read sed file 
sed = readfiles.read_skirt_sed_file(galaxy_name, galaxy_type, redshift, directory_name = "voronoi_1e6", inclination = "0")

--------------  m12i_res7100_md  --------------


In [ ]:
from pprint import pprint

import astropy.units as units


def calculate_coordinates_from_fits_file(hdul):
    
    # Read the header and the data 
    fits_data = hdul[0].data    
    header = hdul[0].header
    
#     pprint(header)
    
    ## Find the coordinates 
    # Extract spatial axis parameters from the header
    crpix1, crval1, cdelt1 = header['CRPIX1'], header['CRVAL1'], header["CDELT1"]  # X-axis
    crpix2, crval2, cdelt2 = header['CRPIX2'], header['CRVAL2'], header["CDELT2"]  # Y-axis
    
    # Generate X and Y coordinates
    x_coords = (np.arange(fits_data.shape[1]) + 1 - crpix1) * cdelt1 + crval1
    y_coords = (np.arange(fits_data.shape[2]) + 1 - crpix2) * cdelt2 + crval2
    #### Convert coordinates to pc
    # arcseconds -> radians 
    arcseconds2radians = 4.84814e-6
    x_coords *= arcseconds2radians
    y_coords *= arcseconds2radians
    # radians -> pc 
    distance = 10 # Mpc
    Mpc2parsec = 1e6 # Mpc -> pc
    x_coords *= distance * Mpc2parsec  # pc
    y_coords *= distance * Mpc2parsec  # pc
    
    # Meshgrid to calculate the coordinate of each pixel.
    X, Y = np.meshgrid(x_coords, y_coords, indexing='ij')
    X = X.ravel() # pc
    Y = Y.ravel() # pc
    
    return X, Y


def seperate_into_annuli(data):
    
    Rgal = np.sqrt(data['x']**2 + data['y']**2)
    bins = np.linspace(0, 20e3, num=101) # The boundaries for 100 annuli's are created
    
    # Calculate digitized bins 
    digitized_indices_for_gas_bins = np.digitize(Rgal, bins)
    
    # Calculate center Rgal for every annuli
    annuli_center_Rgas = (bins[:-1] + bins[1:])/2  
    
    return digitized_indices_for_gas_bins, annuli_center_Rgas 

def half_light_radius(band, file_names, plot=False):
    
    # Read the fits emission file 
    hdul = fits.open(file_names['i0_total_fits'])
    fits_data = hdul[0].data
    
    # Read the wavelengths file 
    wavelengths_unfiltered = np.loadtxt(file_names['skirt_wavelengths'])[:,0] * units.micron
    
    if plot:
        plt.hist(np.log10(wavelengths_unfiltered.to(units.nm).value), bins=100)
        plt.axvline(np.log10(band['max'].value), color='red', linestyle='--', label='Max')
        plt.axvline(np.log10(band['min'].value), color='orange', linestyle='--', label='Min')
        plt.legend()
        plt.show()

    # Find the indices of the wavelengths inside the band 
    condition1 = wavelengths_unfiltered <= band['max']
    condition2 = wavelengths_unfiltered >= band['min']
    indices = np.where(condition1 & condition2)[0]
    # Filter the wavelengths 
    wavelengths = wavelengths_unfiltered[indices].value
    
    # Filter the fits file 
    fits_data = fits_data[indices,:,:]
    
    # Integrate the emission data 
    integrated_data = np.trapz(fits_data, wavelengths, axis=0) # W/m²/arcsec²
    log_integrated_data = np.log10(integrated_data)

    # Create df
    X, Y = calculate_coordinates_from_fits_file(hdul = hdul)
    size_of_a_rectangle = (Y[1] - Y[0])**2 * constants.pc2m**2 # m2 -- The size is the same 
    data = pd.DataFrame(np.array([X,Y,integrated_data.ravel()]).T, columns=["x", "y", "flux"]) 
    data["luminosity"] = data["flux"] * size_of_a_rectangle # Watts
    # Luminosity bulunmadan da sadece flux ile half light radius bulunulabilirdi. Her pixel icin alan sabit oldugunda, flux 
    # sabit bir sayi ile carpiliyor. Dolayisiyla, flux veya luminosity kullanildiginda degisen bir sey olmuyor.
    
    if plot:
        plt.hist2d(data['x'], data['y'], bins=[1024, 1024], weights=np.log10(data['flux']), cmap='inferno')
        plt.colorbar(label='(W/m²/arcsec²)')
        plt.xlabel('X (pc)')
        plt.ylabel('Y (pc)')
        plt.show()    
    
    
    # Data is created. Now seperate data into annuli
    digitized_indices_for_gas_bins, annuli_center_Rgas = seperate_into_annuli(data = data)
    
    # Calculate flux in each annuli
    luminosity_in_annuli = []
    for annuli_number in range(min(digitized_indices_for_gas_bins), max(digitized_indices_for_gas_bins)):
        indices = np.where(digitized_indices_for_gas_bins == annuli_number)
        luminosity_in_annuli.append(sum(data.iloc[indices]['luminosity']))
    
    if plot: 
        plt.scatter(annuli_center_Rgas, luminosity_in_annuli)
        plt.xlabel("Rgal [pc]")
        plt.ylabel("Luminosity [Watts]")
        plt.show()
        
    # Find half light radius
    total_luminosity = sum(data["luminosity"])
    integrated_luminosity_upto_certain_r = 0
    for i in range(len(luminosity_in_annuli)):
        integrated_luminosity_upto_certain_r += luminosity_in_annuli[i]
        if integrated_luminosity_upto_certain_r > total_luminosity/2:
            R_half_life = annuli_center_Rgas[i] # [pc] half light radius is where the half of the total luminosity is reached.
            break
    
    return R_half_life 




# R_half_life = half_light_radius(
#     band = half_light_wavelength_bands['visible'], 
#     file_names = file_names
# ) # pc

# disk_pressure_calculator(
#     band = half_light_wavelength_bands['visible'],
#     file_names = file_names,
# )



In [ ]:
def filter_particles(particles_df, condition):
    
    r = np.sqrt(particles_df['x']**2 + particles_df['y']**2)
    conditions = (r < condition['r_max']) & (abs(particles_df['z']) < condition['z_max'])
    
    return particles_df[conditions].copy()


def disk_pressure_calculator(gas_particles, star_particles, filtering_condition, band, file_names, is_plot=False):
        
    # Filter the particles
    gas_particles = filter_particles(particles_df = gas_particles, condition = filtering_condition)
    star_particles = filter_particles(particles_df = star_particles, condition = filtering_condition)
    
    
    if is_plot:
        particles = star_particles.copy()
        Rmax = 20e3
        plt.hist2d(
            x = particles['x'],
            y = particles['y'],
            bins = 500,
            weights = particles['mass'],
            norm=matplotlib.colors.LogNorm(),        
            range = [[-Rmax, Rmax], [-Rmax, Rmax]]
        )
        plt.xlabel('x [pc]')
        plt.ylabel('y [pc]')
        plt.show()

        plt.hist2d(
            x = particles['x'],
            y = particles['z'],
            bins = 500,
            weights = particles['mass'],
            norm=matplotlib.colors.LogNorm(),        
            range = [[-Rmax, Rmax], [-Rmax, Rmax]]
        )    
        plt.xlabel('x [pc]')
        plt.ylabel('z [pc]')
        plt.show()
    
    
    #### Calculate the pressure 
    area = np.pi * filtering_condition['r_max']**2 # pc2
#     volume = area * filtering_condition['z_max'] #pc3 # Not used now. Ellison's method is used
    ## Column Density calculation
    sigma_gas = sum(gas_particles['mass']) / area # Msolar / pc2
    ## Star Density calculation
#     rho_star_dtolgay = sum(star_particles['mass']) / volume # Msolar / pc3 # Not used now. Ellison's method is used
    sigma_star = sum(star_particles['mass']/ area) # Msolar / pc2
    # Calculate the half light radius 
    R_half_light = half_light_radius(band, file_names, plot=is_plot) # pc   
    # Rstar eqn. 4. in Ellison
    R_star = R_half_light / 1.68 # [pc]
    rho_star = sigma_star / (0.54 * R_star) # [Msolar / pc2] eqn. 3 in Ellison
    
    
    print(f"sigma_gas: {sigma_gas}")
    print(f"rho_star: {rho_star}")

    ### Disk pressure calculation
    ## Convering to SI units 
    sigma_gas_unit_converted = sigma_gas * constants.Msolar2kg / constants.pc2m**2
    rho_star_unit_converted = rho_star * constants.Msolar2kg / constants.pc2m**3
    ## Gas pressure 
    Pgas = np.pi * constants.gravitational_constant * sigma_gas_unit_converted**2 / 2 # [N/m2 = J/m3]
    Pgas *= 1 / constants.kb / constants.m2cm**3 # [K/cm3]
    
    ## Star pressure 
    # Assuming constant σ_z 
    velocity_dispersion_in_z_direction = 11*1e3 # 11 km/s -- Ellison 2024
    Pstar = sigma_gas_unit_converted * np.sqrt(2 * constants.gravitational_constant * rho_star_unit_converted) * velocity_dispersion_in_z_direction # [N/m2]
    Pstar *= 1 / constants.kb / constants.m2cm**3 # [K/cm3] 
    
    ## Total pressure 
    Ptotal = Pgas + Pstar # [K/cm3]
    
    return Pgas, Pstar, Ptotal


half_light_wavelength_bands = {
    "visible": {
        "max": 700 * units.nm, # nm 
        "min": 400 * units.nm, # nm
    },
    "infrared": {
        "max": 1 * units.mm,    # mm
        "min": 780 * units.nm,  # nm
    },
}

conditions = {
    "20kpc": {
        "r_max" : 20e3, # pc
        "z_max" : 2e3, # pc 
    },
    "10kpc": {
        "r_max": 10e3, # pc
        "z_max": 2e3, # pc 
    },
}

Pgas, Pstar, Ptotal = disk_pressure_calculator(
    gas_particles = gas, 
    star_particles = star, 
    filtering_condition = conditions['10kpc'], 
    band = half_light_wavelength_bands['visible'],
    file_names = file_names,
    is_plot = False, 
)


In [ ]:
star.keys()

In [8]:
import astropy.units as units
import galactic_props_functions as galactic_prop_funcs

half_light_wavelength_bands = {
    "visible": {
        "max": 700 * units.nm, # nm 
        "min": 400 * units.nm, # nm
    },
    "infrared": {
        "max": 1 * units.mm,    # mm
        "min": 780 * units.nm,  # nm
    },
}

conditions = {
    "20kpc": {
        "r_max" : 20e3, # pc
        "z_max" : 2e3, # pc 
    },
    "10kpc": {
        "r_max": 10e3, # pc
        "z_max": 2e3, # pc 
    },
}



Pgas, Pstar, Ptotal = galactic_prop_funcs.disk_pressure_calculator(
    gas_particles = gas, 
    star_particles = star, 
    filtering_condition = conditions['10kpc'], 
    band = half_light_wavelength_bands['visible'],
    file_names = file_names,
    is_plot = False, 
)

print(Pgas, Pstar, Ptotal)

/fs/lustre/scratch/RAID-CITA/dtolgay/FIRE/post_processing_fire_outputs/skirt/python_files/analyze_hden_metallicity_turbulence_isrf_radius/create_galactic_properties/galactic_props_functions.py:89: RuntimeWarning: divide by zero encountered in log10
  log_integrated_data = np.log10(integrated_data)


sigma_gas: 31.464291554547962
rho_star: 0.10223578717605913
33047.98110172319 50660.72018662119 83708.70128834438
